# Item 14: Prefer Exceptions to Returning None

- When writing utility functions, there's a draw for Python programmers to give special meaning to the return value of None. It seems to makes sense in some cases. For example, say you want a helper function that divides one number by another. In the case of dividing by zero, returning None seems natural because the result is undefined.

In [1]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

- Code using this function can interpret the return value accordingly.

In [2]:
result = divide(x, y)
if result is None:
    print('Invalid inputs')

NameError: name 'x' is not defined

- What happens when the numerator is zero? That will cause the return value to also be zero. This can cause problems when you evaluate the result in a condition like an if statement. You may accidentally look for any False equivalent value to indicate errors instead of only looking for None 

In [3]:
x, y = 0, 5
result = divide(x, y)
if not result:
    print('Invalid inputs') # This is wrong!

Invalid inputs


- This is a common mistake in Python code when None has special meaning. This is why returning None from a function is error prone. There are two ways to reduce the chance of such errors.

- The first way is to split the return value into a two-tuple. The first part of the tuple indicates that the operation was a success or failure. The second part is the actual result that was computed.

In [4]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

- Callers of  this function have to unpack the tuple. That forces them to consider the status part of the tuple instead of just looking at the result of division.

In [5]:
success, result = divide(x, y)
if not success:
    print('Invalid inputs')

- The problem is that callers can easily ignore the first part of the tuple(using the underscore variable name, a Python convention for unused variables). The resulting code doesn't look wrong at first glance. This is as bad as just returning None.

In [6]:
_, result = divide(x, y)
if not result:
    print('Invalid inputs')
    

Invalid inputs


- The second, better way to reduce these errors is to never return None at all. Instead, raise an exception up to the caller and make them deal with it. Here, I turn a ZeroDivisionError into a ValueError to indicate to the caller that the input values are bad:

In [7]:
def divide(a, b):
    try:
        return a / b 
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e
        

- Now the caller should handle the exception for the invalid input case(this behavior should be documented). The caller no longer requires a condition on the return value of the function. If the function didn't raise an exception, then the return value must be good. The outcome of exception handling is clear.

In [8]:
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 2.5


## Things to Remember

- Functions that return None to indicate special meaning are error prone because None and other values(e.g., zero, the empty string) all evaluate to False in conditional expressions.

- Raise exceptions to indicate special situations instead of returning None. Expect the calling code to handle exceptions properly when they're documented.

# Item 15: Know How Closures Interact with Variable Scope

- Say you want to sort a list of numbers but prioritize one group of numbers to come first. This pattern is useful when you're rendering a user interface and want important messages or exceptional events to be displayed before everything else.

- A common way to do this is to pass a helper function as the key argument to a list's sort method. The helper's return value will be used as the value for sorting each item in the list. The helper can check whether the given item is in the important group and can vary the sort key accordingly.

In [9]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

- This function works for simple inputs.

In [10]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


- There are three reasons why this function operates as expected:

* Python supports closures: functions that refer to variables from the scope in which they were defined. This is why the helper function is able to access the group argument to sort_priority.

* Functions are first-class objects in Python, meaning you can refer to them directly, assign them to variables, pass them as arguments to other functions, compare them in expressions and if statements, etc. This is how the sort method can accept a closure function as the key argument.

* Python has specific rules for comparing tuples. If first compares items in index zero, then index one, then index two, and so on. This is why the return value from the helper closure causes the sort order to have two distinct groups.

- *start again with*
- It’d	be	nice	if	this	function	returned	whether	higher-priority